In [1]:
from crewai import Crew, Task, Agent
from dotenv import load_dotenv
import os
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

/Users/ualguest/Desktop/Projects/CrewAI/venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [3]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [4]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [5]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [8]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [9]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [10]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

financial_trading_crew = Crew(
    agents=[data_analyst_agent, trading_strategy_agent, execution_agent, risk_management_agent],
    tasks=[data_analysis_task, strategy_development_task, execution_planning_task, risk_assessment_task],
    manager_llm=ChatOpenAI(model="gpt-4-turbo", temperature=0.5),
    process=Process.hierarchical,
    verbose=True
)


In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.
# Agent: Data Analyst
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


# Agent: Data Analyst
## Thought: Thought: To monitor and analyze real-time market data for AAPL, I need to find a reliable source that provides continuous updates on financial metrics such as stock prices, volume, and historical trends. Websites like Bloomberg, Yahoo Finance, or Google Finance often provide this information.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"real-time AAPL stock data site:finance.yahoo.com\"}"
## Tool Output: 

Search results: Title: Apple Inc. (AAPL): Historical Data
Link: https://finance.yahoo.com/quote/AAPL/history/
Snippet: Dis

In [16]:
from IPython.display import Markdown
Markdown(result.raw)

**Risk Analysis and Mitigation Strategies for AAPL Trading Plans**

**1. Pre-Earnings Momentum Trading:**
   - **Risk**: Volatility and downward trend risk before earnings, which may affect the momentum strategy.
   - **Mitigation**: Implement tighter stop-loss orders to minimize potential losses and execute staggered purchasing to average down the buying price if necessary.

**2. Earnings Straddle:**
   - **Risk**: High cost of buying both call and put options with the risk of options expiring worthless if AAPL’s price movement is minimal post-earnings.
   - **Mitigation**: Monitor and analyze implied volatility to decide if the potential price move justifies the cost of the options. Set up the straddle only under favorable conditions.

**3. Technical Analysis Integration:**
   - **Risk**: Over-reliance on indicators like RSI and moving averages which might provide misleading signals.
   - **Mitigation**: Supplement technical analysis with fundamental analysis and ensure technical setups are supported by current market conditions. Utilize both short-term and long-term moving averages to capture different market perspectives.

**4. Volatility Monitoring:**
   - **Risk**: Mispricing of options influenced by unexpected spikes in implied volatility, impacting both trading profits and strategy effectiveness.
   - **Mitigation**: Use dynamic hedging and consider options strategies that capitalize on high volatility periods, such as iron condors.

**5. Economic and Market Sentiment Analysis:**
   - **Risk**: Incorrect interpretation of economic indicators and market sentiment leading to inappropriate trading decisions.
   - **Mitigation**: Employ multiple sentiment analysis tools and review economic indicators in relation to historical market reactions under similar circumstances to refine strategy.

**Conclusion:**
This analysis covers the key risks and provides strategic mitigations to guide execution of the proposed AAPL trading strategies. Constant review and adaptation to market conditions are recommended to ensure alignment with overall risk management objectives and capitalization on trading opportunities.